# Adaptive Huffman Algorithm

Adaptive Huffman Coding (Algorithm M) in Python
# --------------------------------------------------
This is a memory-efficient adaptive Huffman coding algorithm converted from C++.
It maintains a tree of frequency classes instead of individual symbols to reduce memory usage.
The tree evolves as input symbols are processed, adapting to symbol frequencies dynamically.

In [26]:
from collections import deque
from typing import List, Optional

# Node class represents a node in the Huffman tree.
class Node:
    def __init__(self, data=None):
        self.occurrences = 0                       # Number of times the symbol(s) occurred
        self.weight = 0                            # Weight used for balancing (depends on frequency)
        self.data: List[int] = [] if data is None else data  # List of symbols in this frequency class
        self.left = None                           # Left child in the tree
        self.right = None                          # Right child in the tree
        self.parent = None                         # Pointer to parent node



## i.Huffman Tree Structure

In [27]:
# Step 1: Initialize the Huffman tree with a root node

def initialize_tree(start_symbol: int) -> Node:
    root = Node()
    root.weight = 1e8  # Large weight to ensure root stays on top
    return root


## ii.To find the leaf node

In [28]:
# Step 2: Utility to find a leaf containing a specific symbol

def find_leaf_by_term(root: Node, dict_node: Node, term: int) -> Node:
    q = deque([root])
    while q:
        current = q.popleft()
        if term in current.data:
            return current
        if current.left:
            q.append(current.left)
        if current.right:
            q.append(current.right)
    return dict_node

## iii.To find leaf by frequency count

In [29]:
# Step 3: Utility to find leaf by frequency count

def find_leaf_by_occurrences(root: Node, target_occurrences: int) -> Optional[Node]:
    q = deque([root])
    while q:
        current = q.popleft()
        if current.occurrences == target_occurrences:
            return current
        if current.left:
            q.append(current.left)
        if current.right:
            q.append(current.right)
    return None


# Update node's weight using its children

def update_weight(node: Node):
    if node.left and node.right:
        node.weight = node.left.weight + node.right.weight
    elif node.left:
        node.weight = node.left.weight
    elif node.right:
        node.weight = node.right.weight
    else:
        node.weight = node.occurrences # if node has no children weight depends on occurrences


# Swap left and right children of a node

def swap_children(node: Node):
    node.left, node.right = node.right, node.left

# Remove a child from its parent node

def remove_child_from_parent(child: Node):
    parent = child.parent
    if not parent:
        return
    if parent.left == child:
        parent.left = None
    else:
        parent.right = None

# Get sibling of a node

def get_sibling(node: Node) -> Optional[Node]:
    if not node.parent:
        return None
    return node.parent.right if node.parent.left == node else node.parent.left

# Exchange positions of two nodes in the tree

def exchange_nodes(left: Node, right: Node):
    if not left or not right or left == right:
        return

    left_parent = left.parent
    right_parent = right.parent

    if not left_parent or not right_parent:
        return

    is_left_left_child = (left_parent.left == left)
    is_right_left_child = (right_parent.left == right)

    left.parent = right_parent
    right.parent = left_parent

    if is_left_left_child:
        left_parent.left = right
    else:
        left_parent.right = right

    if is_right_left_child:
        right_parent.left = left
    else:
        right_parent.right = left

    # Update children’s parent pointers
    for child in [left.left, left.right]:
        if child:
            child.parent = left
    for child in [right.left, right.right]:
        if child:
            child.parent = right

## iv.Rebalancing

In [30]:
# Step 4: Rebalance the tree as needed to maintain structure

def shift_up(root: Node, node: Node):
    curr = node
    while curr != root and curr is not None:
        # Update weight of current node
        if curr.left and curr.right:
            curr.weight = curr.left.weight + curr.right.weight
        else:
            curr.weight = curr.occurrences * curr.weight

        # Perform rotations if imbalance is detected
        if curr.parent and get_sibling(curr) and curr.weight > get_sibling(curr).weight + 1:
            if curr.parent.parent and get_sibling(curr.parent) and curr.weight > get_sibling(curr.parent).weight:
                q = curr.parent.parent
                old_parent = curr.parent
                exchange_nodes(curr, get_sibling(curr.parent))
                swap_children(q)
                update_weight(old_parent)
        curr = curr.parent

## v.For next symbol
-- if only already seen -- find leaf with frequency count
-- else new node

In [31]:
# Step 5: Update the Huffman tree with the next symbol

def update_tree(root_ref: List[Node], dict_node: Node, next_term: int):
    root = root_ref[0]
    p = find_leaf_by_term(root, dict_node, next_term)
    q = find_leaf_by_occurrences(root, p.occurrences + 1)

    if q:
        if p != dict_node:
            p.data.remove(next_term)
            p.weight -= p.occurrences
        q.data.append(next_term)
        q.weight += q.occurrences
        if not p.data and p != dict_node:
            remove_child_from_parent(p)
        elif get_sibling(p):
            shift_up(root, get_sibling(p))
    else:
        new_node = Node()
        new_class = Node([next_term])
        new_class.occurrences = p.occurrences + 1
        new_class.weight = new_class.occurrences

        new_node.left = p
        new_node.right = new_class
        new_node.weight = p.weight + new_class.weight
        new_class.parent = new_node

        if p.parent:
            if p.parent.left == p:
                p.parent.left = new_node
            else:
                p.parent.right = new_node
            new_node.parent = p.parent
        else:
            root_ref[0] = new_node  # New root after restructuring

        p.parent = new_node

        if p != dict_node:
            p.data.remove(next_term)
            p.weight -= p.occurrences

        if not p.data and p != dict_node:
            remove_child_from_parent(p)
        elif get_sibling(p):
            shift_up(root, get_sibling(p))
        shift_up(root, new_node)

## vi. Visualization of Final Tree

In [32]:
# Step 6 (Updated): Visual tree printing in a binary tree format

def print_bt(prefix: str, node: Optional[Node], is_left: bool):
    if node is not None:
        print(prefix, end="")
        print("├── " if is_left else "└── ", end="")
        print(f"Occurrences: {node.occurrences}, Data: {node.data}, Weight: {node.weight}")

        new_prefix = prefix + ("│   " if is_left else "    ")
        print_bt(new_prefix, node.left, True)
        print_bt(new_prefix, node.right, False)

def print_bt_root(node: Node):
    print_bt("", node, False)


Call driver Function

In [34]:
# Step 7: Main function to demonstrate encoding process

def main():
    # Example data stream (simulate pixel values or symbols)
    data = [1, 2, 5, 2, 4, 1, 2, 5, 3, 5, 0, 1]

    # Initialize tree with the first symbol
    tree = initialize_tree(data[0])
    dict_node = tree  # Initially, the tree only has a dictionary node
    root_ref = [tree]  # Use list for mutable reference to root

    # Process each symbol in the stream
    for value in data:
        update_tree(root_ref, dict_node, value)
        print_tree(root_ref[0])
        print("Next Loop\n")

# Entry point
if __name__ == '__main__':
    main()


Occurrences: 0, Data: [], Weight: 100000001.0
Occurrences: 0, Data: [], Weight: 100000000.0
Occurrences: 1, Data: [1], Weight: 1
Next Loop

Occurrences: 0, Data: [], Weight: 100000001.0
Occurrences: 0, Data: [], Weight: 100000000.0
Occurrences: 1, Data: [1, 2], Weight: 2
Next Loop

Occurrences: 0, Data: [], Weight: 100000001.0
Occurrences: 0, Data: [], Weight: 100000000.0
Occurrences: 1, Data: [1, 2, 5], Weight: 3
Next Loop

Occurrences: 0, Data: [], Weight: 100000001.0
Occurrences: 0, Data: [], Weight: 100000000.0
Occurrences: 0, Data: [], Weight: 6
Occurrences: 1, Data: [1, 5], Weight: 2
Occurrences: 2, Data: [2], Weight: 4
Next Loop

Occurrences: 0, Data: [], Weight: 100000001.0
Occurrences: 0, Data: [], Weight: 100000000.0
Occurrences: 0, Data: [], Weight: 7
Occurrences: 1, Data: [1, 5, 4], Weight: 3
Occurrences: 2, Data: [2], Weight: 4
Next Loop

Occurrences: 0, Data: [], Weight: 100000001.0
Occurrences: 0, Data: [], Weight: 100000000.0
Occurrences: 0, Data: [], Weight: 14
Occurre